# Running the fleet of Virtual Wind Turbines and Edge Devices

**SageMaker Studio Kernel**: Data Science

After visualizing the data and training/optimizing/packaging the Anomaly detection model, its time to deploy it and test your virtual fleet. In this exercise you will run a local application written in Python3 that simulates 5 Wind Turbines and 5 edge devices. The SageMaker Edge Agent is deployed on the edge devices.

Here you'll be the **Wind Turbine Farm Operator**. It's possible to visualize the data flowing from the sensors to the ML Model and analyze the anomalies. Also, you'll be able to inject noise (pressing some buttons) in the data to simulate potential anomalies with the equipment.

<table border="0" cellpading="0">
    <tr>
        <td align="center"><b>STEP-BY-STEP</b></td>
        <td align="center"><b>APPLICATION ARCHITECTURE</b></td>
    </tr>
    <tr>
        <td><img src="../imgs/EdgeManagerWorkshop_Macro.png" width="500px"></img></td>
        <td><img src="../imgs/EdgeManagerWorkshop_App.png"  width="500px"></img></td>
    </tr>
</table>  

The components of the applicationare:
  - Simulator:
      - [Simulator](app/ota.py): Program that launches the virtual wind turbines and the edge devices. It uses Python Threads to run all the 10 processes
      - [Wind Farm](app/windfarm.py): This is the application that runs on the edge device. It is reponsible for reading the sensors, invoking the ML model and analyzing the anomalies  
  - Edge Application:
      - [Turbine](app/turbine.py): Virtual Wind Turbine. It reads the raw data collected from the 3D Prited Mini Turbine and stream it as a circular buffer. It also has a graphical representation in **IPython Widgets** that is rendered by the Simulator/Dashboard.
      - [Over The Air](app/ota.py): This is a module integrated with **IoT Jobs**. In the previous exercise you created an IoT job to deploy the model. This module gets the document process it and deployes the model in each edge device and loads it via SageMaker Edge Manager.
      - [Edge client](app/edgeagentclient.py): An abstraction layer on top of the **generated stubs** (proto compilation). It makes it easy to integrate **Wind Farm** with the SageMaker Edge Agent

### Install some dependencies

In [ ]:
!apt-get -y update && apt-get -y install build-essential procps
!pip install -U grpcio-tools grpcio protobuf
!pip install paho-mqtt
!pip install ipywidgets

### Download the sensors data (streamed by the virtual wind turbines)

In [ ]:
!mkdir -p data
!curl https://aws-ml-blog.s3.amazonaws.com/artifacts/monitor-manage-anomaly-detection-model-wind-turbine-fleet-sagemaker-neo/dataset_wind_turbine.csv.gz -o data/dataset_wind.csv.gz

In [ ]:
import boto3
import tarfile
import os
import stat
import io

In [ ]:
project_name='<<PROJECT_NAME>>'

sm_client = boto3.client('sagemaker')
project_id = sm_client.describe_project(ProjectName=project_name)['ProjectId']
bucket_name = 'sagemaker-wind-turbine-farm-%s' % project_id

agent_config_package_prefix = 'wind_turbine_agent/config.tgz'
agent_version = '1.20210512.96da6cc'
agent_pkg_bucket = 'sagemaker-edge-release-store-us-west-2-linux-x64'

### Prepare the edge devices
 1. First download the deployment package that contains the IoT + CA certificates and the configuration file of the SageMaker Edge Agent. 
 2. Then, download the SageMaker Edge Manager package and complete the deployment process.
 
 > You can see all the artifacts that will be loaded/executed by the virtual Edge Device in **agent/**

In [ ]:
if not os.path.isdir('agent'):
    s3_client = boto3.client('s3')

    # Get the configuration package with certificates and config files
    with io.BytesIO() as file:
        s3_client.download_fileobj(bucket_name, agent_config_package_prefix, file)
        file.seek(0)
        # Extract the files
        tar = tarfile.open(fileobj=file)
        tar.extractall('.')
        tar.close()    

    # Download and install SageMaker Edge Manager
    agent_pkg_key = 'Releases/%s/%s.tgz' % (agent_version, agent_version)
    # get the agent package
    with io.BytesIO() as file:
        s3_client.download_fileobj(agent_pkg_bucket, agent_pkg_key, file)
        file.seek(0)
        # Extract the files
        tar = tarfile.open(fileobj=file)
        tar.extractall('agent')
        tar.close()
        # Adjust the permissions
        os.chmod('agent/bin/sagemaker_edge_agent_binary', stat.S_IXUSR|stat.S_IWUSR|stat.S_IXGRP|stat.S_IWGRP)

### Finally, create the SageMaker Edge Agent client stubs, using the protobuffer compiler

In [ ]:
!python3 -m grpc_tools.protoc --proto_path=agent/docs/api --python_out=app/ --grpc_python_out=app/ agent/docs/api/agent.proto

# Simulating The Wind Turbine Farm
Now its time to run our simulator and start playing with the turbines, agents and with the anomalies
 > After clicking on **Start**, each turbine will start buffering some data. It takes a few seconds but after completing this process, the application runs in real-time   
 > Try to press some buttons while the simulation is running, to inject noise in the data and see some anomalies  


In [ ]:
import sys
sys.path.insert(1, 'app')
import windfarm
import edgeagentclient
import turbine
import simulator
import ota
import boto3
from importlib import reload

reload(simulator)
reload(turbine)
reload(edgeagentclient)
reload(windfarm)
reload(ota)

iot_client = boto3.client('iot')

mqtt_host=iot_client.describe_endpoint(endpointType='iot:Data-ATS')['endpointAddress']
mqtt_port=8883

!mkdir -p agent/logs && rm -f agent/logs/*
simulator = simulator.WindTurbineFarmSimulator(5)
simulator.start()

farm = windfarm.WindTurbineFarm(simulator, mqtt_host, mqtt_port)
farm.start()

simulator.show()

 > If you want to experiment with the deployment process, with the wind farm running, go back to [Exercise #2](02%20-%20Training%20with%20Pytorch.ipynb#Building-the-Deployment-Package-SageMaker-Edge-Manager) (in the last cells), replace the variable **model_version** by the constant (string) '2.0' in the Json document used by the IoT Job. Then, create a new IoT Job to simulate how to deploy new versions of the model. Go back to this exercise to see the results.

In [ ]:
farm.halt()

----

## Cleanup
Run the next cell only if you already finished exploring/hacking the content of the workshop.  
This code will delete all the resouces created so far, including the **SageMaker Project** you've created

In [ ]:
import boto3
import time
from shutil import rmtree

iot_client = boto3.client('iot')
sm_client = boto3.client('sagemaker')
s3_resource = boto3.resource('s3')

policy_name='WindTurbineFarmPolicy-%s' % project_id
thing_group_name='WindTurbineFarm-%s' % project_id
fleet_name='wind-turbine-farm-%s' % project_id

# Delete all files from the S3 Bucket
s3_resource.Bucket(bucket_name).objects.all().delete()

# now deregister the devices from the fleet
resp = sm_client.list_devices(DeviceFleetName=fleet_name)
devices = [d['DeviceName'] for d in resp['DeviceSummaries']]
if len(devices) > 0:
    sm_client.deregister_devices(DeviceFleetName=fleet_name, DeviceNames=devices)

# now deregister the devices from the fleet
for i,cert_arn in enumerate(iot_client.list_targets_for_policy(policyName=policy_name)['targets']):
    for t in iot_client.list_principal_things(principal=cert_arn)['things']:
        iot_client.detach_thing_principal(thingName=t, principal=cert_arn)
    iot_client.detach_policy(policyName=policy_name, target=cert_arn)
    certificateId = cert_arn.split('/')[-1]
    iot_client.update_certificate(certificateId=certificateId, newStatus='INACTIVE')
    iot_client.delete_certificate(certificateId=certificateId)

iot_client.delete_role_alias(roleAlias='SageMakerEdge-%s' % fleet_name)
iot_client.delete_thing_group(thingGroupName=thing_group_name)

if os.path.isdir('agent'): rmtree('agent')
sm_client.delete_project(ProjectName=project_name)

Mission Complete!  
Congratulations! :)